In [ ]:
#This code is used to make real time decisions every 6 seconds

In [6]:
import requests
import time
import datetime
import threading
import statistics
from multiprocessing import Process
import json
import sqlalchemy as db
import matplotlib.pyplot as plt
import pylab
column = db.Column
integer = db.Integer
floats = db.Float
string = db.String


In [9]:
#this class is used to compute automated live investment decisions of real-time data. 
#Instances of this class can be created for different currency pairs 
class EUR_USD:
      def __init__(self):
          #agg_data is a list for compliling items computed every 6 minutes
          self.agg_data = []
          #list to store investment decision history made every 6 minutes
          self.decision_history = []
            
      #this method creates a database and table    
      def create_db_and_table(self):
         print('start')
         #create db and open connection
         self.engine = db.create_engine('sqlite:///forex.db')
         conn = self.engine.connect()
         metadata = db.MetaData(self.engine)
         #create table
         table_name = 'EUR_USD'     
         self.currency_pair = table_name.replace('_','/')
         self.table = db.Table(
         table_name, metadata, 
          column('ID', integer, primary_key = True, nullable=False),  
          column('RATE_AS_BID', floats, nullable=False), 
          column('RATE_AS_ASK', floats, nullable=False), 
          column('CURRENCY_PAIR', string, nullable=False), 
          column('TIMESTAMP_FROM_SOURCE', integer, nullable=False), 
          column('TIMESTAMP_AT_INSERTION', integer, nullable=False), 
          )
         metadata.create_all(self.engine)
         #print("table created successfully")
        
      #this method calls the forex api to return a list of objects      
      def get_live_data(self):
          #call api for currency pair
          url = 'https://api.polygon.io/v1/conversion/'+self.currency_pair+'?amount=100&precision=2&apiKey=beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'
          self.data = requests.get(url).json()
          return self.data
    
      #this method stores data retreived every second into table
      def store_data(self):   
         #create connection to db
          conn1 = self.engine.connect()
          # use inbuilt function to get timestamp at insertion
          ts = round(datetime.datetime.now().timestamp())
          #loop through api response object
          for key, value in self.data.items():
               ins = db.insert(self.table).values(
               RATE_AS_BID = self.data['last']['bid'],
               RATE_AS_ASK = self.data['last']['ask'],
               CURRENCY_PAIR = self.data['from'] + '/' + self.data['to'] ,
               TIMESTAMP_FROM_SOURCE = self.data['last']['timestamp'],
               TIMESTAMP_AT_INSERTION = ts
               )
               conn1.execute(ins)
          #print("Inserted successfully") 
        
      #this method selects rate_as_bid as list_of_price every 6 minutes from the currency table in database. returns a list of prices  
      def select_prices_6mins(self):
        #connect to database
        conn2 = self.engine.connect()
        #write query extracting last 360 items inserted the table
        s = db.select(self.table.c.RATE_AS_BID).order_by(self.table.c.ID.desc()).limit(360)
        result = conn2.execute(s).all()
        self.price_list = [item[0] for item in result]
        return self.price_list
        
      #this method takes agg_list and key as parameter and computes average of grannular prices every 6 minutes. returns average as floating number 
      def calculate_avg(self, agg_list, key):
        #if parameter is a list
        if key == None:
            values = agg_list
        else: #parameter is a list of dictionaries
            values = list(map(lambda items: float(items.get(key)), agg_list))
        sum = 0
        no_of_items = len(values)
        for item in values:
            sum += float(item)
        avg = sum/no_of_items
        return avg
    
      #this method takes mean, agg_list and key as parameters and computes stdev of grannular prices every 6 minutes. returns stdev as floating number 
      def calculate_std_dev(self, mean, agg_list, key):
        #if parameter is a list
        if key == None:
            values = agg_list
            #print(values)
        else: #parameter is a list of dictionaries
            values = list(map(lambda items: float(items.get(key)), agg_list))
            #print(values)
        sum = 0
        no_of_items = len(values)-1
        for item in values:
            dev_mean = float(item) - float(mean)
            dev_mean_squared = dev_mean**2
            sum += dev_mean_squared
            stdev = (sum/no_of_items)**0.5
        return stdev
      
      #this method takes agg_list and key as parameters and computes average of grannular prices every 6 minutes using statistics library. returns avg_library as floating number
      def calculate_average_library(self, agg_list, key):
        #if parameter is a list
        if key == None:
           return statistics.mean(agg_list)
        else: #parameter is a list of dictionaries
           #get values of price only as a list
           values = list(map(lambda items: items.get(key), agg_list))
           return statistics.mean(values)
    
      #this method takes agg_list and key as parameters and compute stdev of grannular prices every 6 minutes using statistics library. returns stdev_library as floating number
      def calculate_stdev_library(self, agg_list, key):
        #if parameter is a list
        if key == None:
           return statistics.stdev(agg_list)
        else: #parameter is a list of dictionaries
           #get values of price only as a list
           values = list(map(lambda items: items.get(key), agg_list))  
           return statistics.stdev(values)
      
      #This method takes agg_list and key as parameters and uses average price computed every minute to calculate 'returns'. Returns 'returns' as floating number
      def calculate_return(self, agg_list, key):
         #calculate number of items in the agg_list
         self.size_of_list = len(agg_list)
         #compute previous price of last item in the list
         self.previous_price = float(agg_list[self.size_of_list - 2][key])
         #compute latest price of items in the list
         self.latest_price = float(agg_list[self.size_of_list - 1][key])
         print('size of aggregate list:', self.size_of_list) 
         #calculate from the second item in the list since first item has no returns
         if self.size_of_list > 1:
            returns = 100*((self.latest_price / self.previous_price) - 1)
            return returns
         else: 
            return None
        
      #This method takes agg_list and key as parameters and computes average of returns by calling the calculate_avg method. Returns 'avg_returns' as floating number   
      def calculate_average_returns(self, agg_list, key):
         #average of returns from the second item in the list since first item has no returns
         if self.size_of_list > 1:
             self.list_items_except_first_item = agg_list[1:]
             #call the calculate_avg method to compute average returns
             average_returns = self.calculate_avg(self.list_items_except_first_item, key)
             return average_returns
         else: 
             return None
            
      #this method takes agg_list and key as parameters and calls the calculate_avg method to compute moving average of 5 data points. Returns moving average as floating number 
      def calculate_moving_average(self, agg_list, key):
         #average of last 5 data points 
         if self.size_of_list > 4:
             #extract list of last 5 items in aggregate list
             list_of_last_5_items = agg_list[-5:]
             self.moving_average = self.calculate_avg(list_of_last_5_items, key)
             #call standard deviation method to compute moving standard deviation
             self.moving_stdev = self.calculate_std_dev(self.moving_average, list_of_last_5_items, key)
             print("mov_avg: ", self.moving_average)
             #print("mov_stdev: ", self.moving_stdev)
             return self.moving_average
         else:
             return None
        
      #This method computes bollinger bands. Returns a dictionary of upper and lower bollinger bands as floating numbers     
      def calculate_bollinger_bands(self):
            #use moving average and stdev to calculate the upper and lower bands
            #buy signals when the price hits the lower band. oversold stock #concept of higher supply, lower price
            #sell signals when the price hits the upper band. overbought stock # concept higher demand, higher price
            #calculate bollinger bands from 5th item in the list since moving average computes for 5 data points
            if self.size_of_list > 4 :
                bollinger_upper = self.moving_average + 1.25*self.moving_stdev
                bollinger_lower = self.moving_average - 1.25*self.moving_stdev

                return {'bollinger_upper': bollinger_upper, 'bollinger_lower': bollinger_lower }
            else:
                return {'bollinger_upper': None, 'bollinger_lower': None }
            
      #This method takes agg_list, no_of_returns based on strategy, and key as parameters. Returns a default decision as string based on number of consecutive +ve or -ve returns      
      def default_decision(self, agg_list, no_of_returns, key):
          positive = 0
          negative = 0
          zeros = 0
          #make decision from 5th item in the list
          if self.size_of_list > 4:
              #extract only values of last n returns in agg list to a return_values list
              return_values = list(map(lambda items: items.get(key), agg_list[-no_of_returns:]))
              #print(return_values)
              #loop through each return value in list of return values
              for return_value in return_values:
                #check if return is poitive
                if return_value > 0:
                  #count positive value
                  positive += 1
                #check if return is negative
                elif return_value < 0:
                  #count negative value
                  negative += 1
                else:
                  #count zero value
                  zeros += 1
              #print("positive: ", positive)
              #print("negative: ", negative)
              #check if no of positive values equals number of return values in list of return values
              if positive == len(return_values):
                #return default decision to sell
                return 'sell'
              elif negative == len(return_values):
                #return default decision to buy
                return 'buy'
                #make no decision
              else:
                return 'do nothing'
            
      #This method returns last action(decision) made as a string      
      def get_last_action(self):
          # loop through decision history backwards
          for item in self.decision_history[:3:-1]:
            #check if item in decision history is not "do nothing"
            if item != 'do nothing':
              print('decision:', item)
              return item
            else: #continue looping until decision is not 'do nothing'
              continue
            #N.B: 'do nothing' is returned only when its the first decision made or no decision has been made
          return 'do nothing'
      
      #This method takes agg_list, no_of_returns and key as parameters returns final decision as a string based on last decision and default decision 
      #N.B: After a sell, you have a buy and after a buy you have a sell
      def get_final_decision(self, agg_list, no_of_returns, key):
          if self.size_of_list > 4:
              #call default decision method to get default decision based on returns
              default_decision = self.default_decision(agg_list, no_of_returns, key)
              print("default_decision: ", default_decision)
              #if the array of decision history is empty then return default decision.
              if len(self.decision_history) < 1:
                    return default_decision
              else: #compare default action with last action
                #call last action method to get last action made
                self.last_action = self.get_last_action()
                print("last_action: ", self.last_action)
                    #if any last action and default action are the same, return 'do nothing'
                if self.last_action == default_decision:
                    return 'do nothing'
                else: #return default action since it is not the same as last action.
                    return default_decision
                
      #This methods computes change in price and returns whether a profit or loss was made as a string
      def calculate_profit_and_loss(self):
          #most recent price - previous price
          #consider short position and long position
          #long position = expectation that what you purchased will rise in value (a previous buy)
          #short position = selling security with expectation that you will purchase
          #it later when value reduces(a previous sell).
          change_in_price = self.latest_price - self.previous_price
          print("change_in_price: ", change_in_price)
          if self.size_of_list > 5:
              #positive change_in_price = gain for long position and loss for short position 
              if change_in_price > 0:
                #short position
                if self.last_action == 'sell':
                    p_or_l = 'loss'
                else: #long position or no decision made previously
                    p_or_l = 'profit'
              #negative change_in_price = gain for short position and loss for long position
              elif change_in_price < 0:
                #short position
                if self.last_action == 'sell':
                    p_or_l = 'profit'
                else: #long position or no decision made previously
                    p_or_l = 'loss' 
              else: #no change_in_price
                p_or_l = 'Breakeven'
              return p_or_l
          else:
                return None
          

      #This method calls other methods to perform actions every second
      def compute_every_second(self):
        time_count_in_seconds = 0
        after_a_day_in_seconds = 86400
        while time_count_in_seconds < after_a_day_in_seconds:
          time.sleep(1)
          #get live data
          data = self.get_live_data()
          #store live data in db
          item = self.store_data()
          time_count_in_seconds += 1
          #print(time_count_in_seconds)
          
      #This method calls other methods to make computations every 6 minutes    
      def compute_every_6min(self):
        time_count_in_seconds_2 = 0
        after_a_day_in_seconds_2 = 86400
        while time_count_in_seconds_2 < after_a_day_in_seconds_2:
          time.sleep(360)
          time_count_in_seconds_2 += 360
            
          #select prices 
          price_list = self.select_prices_6mins()
          #print(price_list)
        
          #calculate avg_price 
          avg_price = self.calculate_avg(price_list, None)
          print("avg_price:", avg_price)
          #calculate avg_price using library
          avg_price_library = self.calculate_average_library(price_list, None)
          print("avg_price using statistics library:", avg_price_library)
            
          #compute std_dev 
          std_dev = self.calculate_std_dev(avg_price, price_list, None)
          print("std_dev:", std_dev)
          #compute std_dev using library
          stdev_library = self.calculate_stdev_library(price_list, None)
          print("std_dev using statistics libary:", stdev_library)
        
          #create object of items computed every 6 minutes
          items = { 'time': int(time_count_in_seconds_2/6),
                    'avg_price': avg_price,
                    'std_dev': std_dev  
          }
        
          #append object to agg_data list to compile object of items
          self.agg_data.append(items)
        
          #compute returns 
          returns = self.calculate_return(self.agg_data,'avg_price')
          print("returns:", returns)
            
          #append object of returns to agg_data list updated every 6 minutes
          self.agg_data[len(self.agg_data)-1]['returns'] = returns
            
          #calculate average stdev
          avg_stdev = self.calculate_avg(self.agg_data, 'std_dev')
          #print("avg_stdev:", avg_stdev)
        
          #append object of avg_stdev to agg_data updated every 6 minutes
          self.agg_data[len(self.agg_data)-1]['avg_stdev'] = avg_stdev
        
          #calculate average returns
          avg_returns = self.calculate_average_returns(self.agg_data, 'returns')
        
          #append object of avg_returns to agg_data updated every 6 minutes
          self.agg_data[len(self.agg_data)-1]['avg_returns'] = avg_returns
          
          #calculate moving average 
          moving_avg = self.calculate_moving_average(self.agg_data,'avg_price')
        
          #append object of moving average to agg_data updated every 6 minutes
          self.agg_data[len(self.agg_data)-1]['moving_avg'] = moving_avg
        
          #calculate bollinger bands 
          result = self.calculate_bollinger_bands()
          bollinger_upper = result['bollinger_upper']
          bollinger_lower = result['bollinger_lower']
        
          #append objects of bollinger bands to agg_data updated every 6 minutes
          self.agg_data[len(self.agg_data)-1]['bollinger_upper'] = bollinger_upper
          self.agg_data[len(self.agg_data)-1]['bollinger_lower'] = bollinger_lower
            
          #make decision on whether to buy or sell, or do nothing
          decision = self.get_final_decision(self.agg_data, 3, 'returns')
          
          #append decicion made to decision history list and create an object of decision in agg_data
          self.decision_history.append(decision)
          print("final decision: ", decision)
          self.agg_data[len(self.agg_data)-1]['decision'] = decision
          
          #compute profit or loss
          profit_or_loss = self.calculate_profit_and_loss()
          print("profit_or_loss: ", profit_or_loss)
          
          #append object of profit or loss to agg_data updated every 6 minutes
          self.agg_data[len(self.agg_data)-1]['profit_or_loss'] = profit_or_loss
              
          #print list of objects updated every 6 minutes 
          print(self.agg_data)
        
        
#create an instance of my currency pair class
eur_usd = EUR_USD()

#call function to create my db and table
createDb = eur_usd.create_db_and_table()

#create threads to call methods
if __name__ == "__main__":
    # creating thread for grannular computation
    t1 = threading.Thread(target=eur_usd.compute_every_second)
    # creating thread for 6 minutes computation
    t2 = threading.Thread(target=eur_usd.compute_every_6min)
    # creating thread for 1 hour computation
    #t3 = threading.Thread(target=eur_usd.compute_every_1hour)
  
    # starting thread 1
    t1.start()
    # starting thread 2
    t2.start()
    t1.join()
    t2.join()
    
  
    


start
avg_price: 0.9163899999999991
avg_price using statistics library: 0.91639
std_dev: 8.894145767627814e-16
std_dev using statistics libary: 0.0
size of aggregate list: 1
returns: None
final decision:  None
change_in_price:  0.0
profit_or_loss:  None
[{'time': 1, 'avg_price': 0.9163899999999991, 'std_dev': 8.894145767627814e-16, 'returns': None, 'avg_stdev': 8.894145767627814e-16, 'avg_returns': None, 'moving_avg': None, 'bollinger_upper': None, 'bollinger_lower': None, 'decision': None, 'profit_or_loss': None}]
avg_price: 0.9163899999999991
avg_price using statistics library: 0.91639
std_dev: 8.894145767627814e-16
std_dev using statistics libary: 0.0
size of aggregate list: 2
returns: 0.0
final decision:  None
change_in_price:  0.0
profit_or_loss:  None
[{'time': 1, 'avg_price': 0.9163899999999991, 'std_dev': 8.894145767627814e-16, 'returns': None, 'avg_stdev': 8.894145767627814e-16, 'avg_returns': None, 'moving_avg': None, 'bollinger_upper': None, 'bollinger_lower': None, 'decisio

avg_price: 0.9163899999999991
avg_price using statistics library: 0.91639
std_dev: 8.894145767627814e-16
std_dev using statistics libary: 0.0
size of aggregate list: 7
returns: 0.0
mov_avg:  0.9163899999999991
default_decision:  do nothing
last_action:  do nothing
final decision:  do nothing
change_in_price:  0.0
profit_or_loss:  Breakeven
[{'time': 60, 'avg_price': 0.9061550000000003, 'std_dev': 0.007247573602232519, 'returns': None, 'avg_stdev': 0.007247573602232519, 'avg_returns': None, 'moving_avg': None, 'bollinger_upper': None, 'bollinger_lower': None, 'decision': None, 'profit_or_loss': None}, {'time': 120, 'avg_price': 0.9109448888888919, 'std_dev': 0.007341015010226982, 'returns': 0.5285948749266467, 'avg_stdev': 0.0072942943062297505, 'avg_returns': 0.5285948749266467, 'moving_avg': None, 'bollinger_upper': None, 'bollinger_lower': None, 'decision': None, 'profit_or_loss': None}, {'time': 180, 'avg_price': 0.9163899999999991, 'std_dev': 8.894145767627814e-16, 'returns': 0.597

avg_price: 0.9163899999999991
avg_price using statistics library: 0.91639
std_dev: 8.894145767627814e-16
std_dev using statistics libary: 0.0
size of aggregate list: 10
returns: 0.0
mov_avg:  0.9163899999999991
default_decision:  do nothing
last_action:  do nothing
final decision:  do nothing
change_in_price:  0.0
profit_or_loss:  Breakeven
[{'time': 60, 'avg_price': 0.9061550000000003, 'std_dev': 0.007247573602232519, 'returns': None, 'avg_stdev': 0.007247573602232519, 'avg_returns': None, 'moving_avg': None, 'bollinger_upper': None, 'bollinger_lower': None, 'decision': None, 'profit_or_loss': None}, {'time': 120, 'avg_price': 0.9109448888888919, 'std_dev': 0.007341015010226982, 'returns': 0.5285948749266467, 'avg_stdev': 0.0072942943062297505, 'avg_returns': 0.5285948749266467, 'moving_avg': None, 'bollinger_upper': None, 'bollinger_lower': None, 'decision': None, 'profit_or_loss': None}, {'time': 180, 'avg_price': 0.9163899999999991, 'std_dev': 8.894145767627814e-16, 'returns': 0.59

In [ ]:
['EUR_USD', 'USD_NGN', 'GBP_USD', 'CNY_USD', 'EUR_GBP', 'CAD_USD', 'EUR_CNY', 'CNY_CAD', 'EGP_USD', 'GBP_INR','INR_USD']